What next? Well I am not sure where I am going with this. To continue with the theme "This is probably a bad idea to do that with iPython" I would like to implement a server/client architecture in our game.

But first, I made a game02.py that we can import easily without having to recopy the whole file in the notebook. I will try to make a gameXX.py file at the end of each notebook.

In [17]:
from game02 import *

In [23]:
game = Game()
game.scene.append(Map())
game.player = Character(100,100)
game.scene.append(game.player)
game.event_handlers[(('type',pygame.KEYDOWN),)] = game.player.handle_keydown


th = threading.Thread(target = game.mainloop)
th.start()

Yay, it works!

Before going into the network stuff, let's change a trivial detail: let's add some clothes to our character. At one point we will actually have two players, it is nice to be able to differentiate them.

Let me introduce a nice little tool: [The Universal LPC Sprite Sheet Character Generator](http://gaurav.munjal.us/Universal-LPC-Spritesheet-Character-Generator/#). In 2013, some sponsors proposed money for the best open source 2D pixel art people could deliver and organized the Liberated Pixel Cup. As a result we got a huge stash of graphics we are free to use. This tool is one of the consequences of that.

So I added two character sheets: [Zeetha](art/LPC/zeetha.png) and [Lex](art/LPC/lex.png). They are not the same size as the original [walk](art/LPC/walk.png) sheet so we will need to do some adaptations. Let's first separate the image loading from the `__init__` function and then let's find the good coordinates for these sheets.

In [20]:
def make_seq(self, img, x,y, ticks=80):
    seq = list()
    seq.append(ticks) # ticks per frame
    for i in range(8):
        seq.append(self.img.subsurface((x+i*64,y,64,64)))
    return seq
    
def load_sequences(self, imgname):
    self.img=pygame.image.load(imgname)
    # Each animation is stored in the anim dict as a list with the following 
    # format: (tick_per_frame, frame1, frame2, ...)
    
    self.dirs={"up":(0,-0.5),"down":(0,0.5), "right":(1,0), "left":(-1,0)}
    self.anim = dict()
    self.cycle_index = 0
    self.cycle_tick = 0

    self.anim["up"] = self.make_seq(self.img, 64, 8*64)
    self.anim["left"] = self.make_seq(self.img, 64, 9*64)
    self.anim["down"] = self.make_seq(self.img, 64, 10*64)
    self.anim["right"] = self.make_seq(self.img, 64, 11*64)

def new_init(self, x,y, filename = "art/LPC/lex.png"):
    self.load_sequences(filename)
    self.current_anim = "up"
    self.current_frames = self.anim[self.current_anim]
    self.pos = [x,y]

Character.__init__ = new_init
Character.load_sequences = load_sequences
Character.make_seq = make_seq
game.refresh()

In [24]:
game.refresh()
game.player.load_sequences("art/LPC/zeetha.png")